## 介绍

本课将涵盖：
- 什么是函数调用及其使用场景
- 如何使用 OpenAI 创建函数调用
- 如何将函数调用集成到应用程序中

## 学习目标

完成本课后，您将了解并掌握：

- 使用函数调用的目的
- 使用 OpenAI 服务设置函数调用
- 为您的应用场景设计有效的函数调用


## 理解函数调用

在本课中，我们希望为我们的教育初创公司构建一个功能，允许用户使用聊天机器人查找技术课程。我们将推荐适合他们技能水平、当前角色和感兴趣技术的课程。

为完成此任务，我们将结合使用：
 - `OpenAI` 为用户创建聊天体验
 - `Microsoft Learn Catalog API` 帮助用户根据请求查找课程
 - `Function Calling` 将用户的查询发送到函数以进行 API 请求。

首先，让我们看看为什么我们首先要使用函数调用：

print("下一次请求中的消息：")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # 从 GPT 获取一个新响应，它可以看到函数响应


print(second_response.choices[0].message)


### 为什么使用函数调用

如果你已经完成了本课程中的其他任何课程，你可能已经理解了使用大型语言模型（LLMs）的强大功能。希望你也能看到它们的一些局限性。

函数调用是 OpenAI 服务的一个功能，旨在解决以下挑战：

响应格式不一致：
- 在函数调用之前，大型语言模型的响应是无结构且不一致的。开发者不得不编写复杂的验证代码来处理输出中的每种变体。

与外部数据集成有限：
- 在此功能之前，很难将应用程序其他部分的数据整合到聊天上下文中。

通过标准化响应格式并实现与外部数据的无缝集成，函数调用简化了开发过程，减少了额外验证逻辑的需求。

用户无法获得诸如“斯德哥尔摩当前天气如何？”之类的答案。这是因为模型的知识仅限于训练数据的时间范围。

让我们看下面的示例来说明这个问题：

假设我们想创建一个学生数据数据库，以便为他们推荐合适的课程。下面我们有两个学生的描述，它们在包含的数据上非常相似。


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

我们想将此发送给大型语言模型（LLM）以解析数据。稍后可以在我们的应用程序中使用这些数据，将其发送到API或存储在数据库中。

让我们创建两个相同的提示，指示LLM我们感兴趣的信息：


我们想将此发送给大型语言模型（LLM），以解析对我们的产品重要的部分。因此，我们可以创建两个相同的提示来指导LLM：


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


在创建这两个提示后，我们将使用 `openai.ChatCompletion` 将它们发送给大型语言模型（LLM）。我们将提示存储在 `messages` 变量中，并将角色分配为 `user`。这是为了模拟用户向聊天机器人发送消息。


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

现在我们可以将这两个请求发送给大型语言模型（LLM），并检查我们收到的响应。


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

尽管提示相同且描述类似，我们仍然可能得到不同格式的 `Grades` 属性。

如果多次运行上述单元格，格式可能是 `3.7` 或 `3.7 GPA`。

这是因为大语言模型接收的是以书面提示形式存在的非结构化数据，并且返回的也是非结构化数据。我们需要一个结构化的格式，以便在存储或使用这些数据时知道预期的内容。

通过使用函数调用，我们可以确保收到结构化的数据。使用函数调用时，LLM 实际上并不会调用或运行任何函数。相反，我们为 LLM 创建一个结构，供其响应时遵循。然后，我们使用这些结构化的响应来确定在我们的应用程序中运行哪个函数。


![函数调用流程图](../../../../translated_images/Function-Flow.083875364af4f4bb.zh.png)


然后我们可以将函数返回的内容发送回大型语言模型（LLM）。LLM随后将使用自然语言来回答用户的查询。


### 使用函数调用的用例

**调用外部工具**  
聊天机器人非常擅长为用户的问题提供答案。通过使用函数调用，聊天机器人可以利用用户的消息来完成某些任务。例如，学生可以请求聊天机器人“给我的导师发送电子邮件，说我需要更多关于这门课程的帮助”。这可以调用函数 `send_email(to: string, body: string)`

**创建 API 或数据库查询**  
用户可以使用自然语言查找信息，这些信息会被转换成格式化的查询或 API 请求。一个例子是老师请求“谁完成了上一次作业”，这可以调用名为 `get_completed(student_name: string, assignment: int, current_status: string)` 的函数

**创建结构化数据**  
用户可以将一段文本或 CSV 文件交给大语言模型，从中提取重要信息。例如，学生可以将一篇关于和平协议的维基百科文章转换成 AI 闪卡。这可以通过调用函数 `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` 来完成


## 2. 创建您的第一个函数调用

创建函数调用的过程包括三个主要步骤：
1. 使用您的函数列表和用户消息调用 Chat Completions API
2. 读取模型的响应以执行操作，即执行函数或 API 调用
3. 使用函数的响应再次调用 Chat Completions API，利用该信息创建对用户的响应。


![函数调用流程](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.zh.png)


### 函数调用的元素

#### 用户输入

第一步是创建一个用户消息。这可以通过获取文本输入的值动态分配，或者你可以在这里分配一个值。如果这是你第一次使用聊天补全 API，我们需要定义消息的 `role` 和 `content`。

`role` 可以是 `system`（创建规则）、`assistant`（模型）或 `user`（最终用户）。对于函数调用，我们将其分配为 `user` 并给出一个示例问题。


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 创建函数。

接下来我们将定义一个函数及该函数的参数。这里我们只使用一个名为 `search_courses` 的函数，但你可以创建多个函数。

**重要**：函数包含在发送给大型语言模型的系统消息中，并且会计入你可用的令牌数量。


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**定义**

函数定义结构有多个层级，每个层级都有其自身的属性。以下是嵌套结构的详细说明：

**顶层函数属性：**

`name` - 我们希望调用的函数名称。

`description` - 这是函数的工作描述。在这里，明确和具体非常重要。

`parameters` - 你希望模型在响应中生成的值和格式的列表。

**参数对象属性：**

`type` - 参数对象的数据类型（通常是 "object"）

`properties` - 模型将在响应中使用的具体值列表。

**单个参数属性：**

`name` - 由属性键隐式定义（例如 "role"、"product"、"level"）

`type` - 该具体参数的数据类型（例如 "string"、"number"、"boolean"）

`description` - 该具体参数的描述。

**可选属性：**

`required` - 列出哪些参数是完成函数调用所必需的数组。


### 调用函数  
定义函数后，我们现在需要在调用 Chat Completion API 时包含它。我们通过向请求中添加 `functions` 来实现。在本例中为 `functions=functions`。

还有一个选项是将 `function_call` 设置为 `auto`。这意味着我们将让 LLM 根据用户消息决定调用哪个函数，而不是自己指定。


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

现在让我们来看一下响应以及它是如何格式化的：

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

你可以看到函数的名称被调用了，并且从用户消息中，LLM能够找到数据来匹配函数的参数。


## 3.将函数调用集成到应用程序中。

在我们测试了来自大型语言模型的格式化响应之后，现在可以将其集成到应用程序中。

### 管理流程

为了将其集成到我们的应用程序中，让我们采取以下步骤：

首先，让我们调用 OpenAI 服务并将消息存储在名为 `response_message` 的变量中。


In [ ]:
response_message = response.choices[0].message

现在我们将定义一个函数，该函数将调用 Microsoft Learn API 以获取课程列表：


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



作为最佳实践，我们将首先查看模型是否想调用某个函数。之后，我们将创建一个可用的函数，并将其与被调用的函数进行匹配。  
然后，我们将获取该函数的参数，并将其映射到来自LLM的参数。  

最后，我们将附加函数调用消息以及由`search_courses`消息返回的值。这为LLM提供了所有所需的信息，以便使用自然语言响应用户。


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



现在我们将发送更新后的消息给大型语言模型，以便我们可以收到自然语言的回复，而不是API的JSON格式回复。


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## 代码挑战

干得好！为了继续学习 OpenAI 函数调用，你可以构建：https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
- 函数的更多参数，可能帮助学习者找到更多课程。你可以在这里找到可用的 API 参数：  
- 创建另一个函数调用，获取学习者的更多信息，比如他们的母语  
- 当函数调用和/或 API 调用未返回任何合适的课程时，创建错误处理机制


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免责声明**：  
本文件由人工智能翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译而成。尽管我们力求准确，但请注意自动翻译可能存在错误或不准确之处。原始语言的文档应被视为权威来源。对于重要信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或误释，我们概不负责。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
